<h1 class="text-center">BCI - Decoding frequency-tagging: a SSVEP-based BCI</h1>
<h2 class="text-center">February, 2022</h2>

<br>

The purpose of this tutorial is to implement a reactive BCI using SSVEP on a dataset collected in our laboratory. You will use MNE to load and pre-process the data and Sklearn+MNE for the classification part. 
</b></div>

- In Section I, exploration data analysis, frequency analysis and epoching using MNE
- In Section II, a first classifier is trained on SNR at stimulation frequencies
- In Section III, another pipeline that uses Canonical Correlation Analysis with sinus templates to learn a spatial filter.
- 📜 The last section (IV) is the evaluation. In Section IV, a more advance pipeline based on Task Related Correlation Analysis that add individual templates from calibration data. You would have 2 weeks to implement a classification pipeline using this classifier.

The code must be completed after each ❓ **Question** ❓. A blank cell with "HERE" appears as a comment in the code. The parameters that do not change the course of the story are accompanied "EDIT ME!" as a comment: you can change them at the time or at the end of the section to see the changes involved.

You can also find some 🔴 HINTS 🔴 with associated links to documentation and usefull functions.

# Install packages
Execute following cell to install the packages.

In [1]:
!pip install -U mne scipy scikit-learn

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 9.0 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 11.9 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 11.2 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 11.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
openvino-dev 2021.4.1 requires numpy<1.20,>=1.16.6, but you have numpy 1.22.0 which is incompatible.
openvino-dev 2021.4.1 requires scipy~=1.5.4, but you have scipy 1.10.0 which is incompatible.


In [ ]:
import mne
import os

import matplotlib.pyplot as plt

import numpy as np

from scipy.stats import ttest_rel
from scipy.signal import welch
from sklearn.cross_decomposition import CCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from TRCA import TRCA

# I - Dataset exploration and pre-processing
In this session we will work with data acquired at ISAE-SUPAERO on Steady States Visually Evoked Potentials (SSVEP). EEG data was collected using 32 Ag/AgCl active electrodes. A 32-channel montage based on the international 10-20 system was used to record the EEG signals with a sampling rate of 500Hz. The EEG device used in this experiment was the Brain Products LiveAmp system. Brain data was recorded using the LabRecorder software and the experimental protocol was implemented using the Psychopy Python library. Events from the experimental paradigm were synchronized with the EEG signal recording using the Lab Streaming Layer (LSL) library.

**The subjects were asked to look at four different stimuli with rectangular shapes**. These stimuli flickered at different frequencies. Because of this difference in frequency, each stimuli elicits a different response in the Primary Visual Cortex, that we can classify in order to know at which target the participants were looking at each trial.

## We load the data and plot the sensor location
The data is presented in [EEGLAB .set format](https://eeglab.org/tutorials/03_Dataset_management/datasets.html). MNE supports data-loading functions in most common file formats in their `mne.io` module, check [here](https://mne.tools/0.18/manual/io.html#id15) for a complete list and link to the corresponding functions.

In [ ]:
data_dir = '/path/to/the/data'
data_file = 'P1_low_100.set'

In [ ]:
# Load data
data_path = os.path.join(data_dir, data_file)
raw_data = mne.io.read_raw_eeglab(data_path, preload=True, verbose=False)

# Show info (dict containing relevant metadata)
print(raw_data.info)

# Display the montage (sensors on the scalp)
plt.rcParams['figure.dpi'] = 150
raw_data.plot_sensors(ch_type='eeg',show_names=True)
plt.show()

## Now let us explore the EEG data.
As previously, the data array has a shape of (channels, time). We use the `get_data()` method to obtain the EEG array.

In [ ]:
data_array = raw_data.get_data()

# Print the shape of the data
print(data_array.shape)

As previously stated, this data was acquired using a sampling frequency of 500Hz. We can recover this parameter from the `info` structure. This will be useful as we progress, but for now let's find out how long our data is in seconds.

In [ ]:
sfreq = raw_data.info['sfreq']  # Sampling frequency
seconds = data_array.shape[-1] // sfreq
print(f'Data duration in seconds: {seconds} (around {seconds // 60} minutes)')

Data can be conveniently plotted from the raw object directly, allowing us some handy operations like filtering the data before displaying

In [ ]:
plt.rcParams['figure.dpi'] = 150
scal = dict(eeg=1e-3)                      # EDIT ME!
raw_data.plot(n_channels=32, scalings=scal,
              start=15, duration=2,             # EDIT ME!
              lowpass=40, highpass=2,          # EDIT ME!
              show_scrollbars=False, show_scalebars=False)
plt.show()

### Let's explore some of the events
In this case, we find the events on an annotation file. Several ways of storing events exist, please refer to [the MNE documentation](https://mne.tools/dev/auto_tutorials/raw/20_event_arrays.html) to learn more about how to interact with different types of events. Here, we use the `events_from_annotations()` function to load them.

In [ ]:
# Annotations are part of the raw object
print(raw_data.annotations)
print()

# We load the events and the event_id
events, event_id = mne.events_from_annotations(raw_data, verbose=False)

# event_id is a dictionary that related each label to their event name
print(event_id)
print()

# The events are a list where each element is a 3-element list. The first element is the onset of the event, and the last one is the label according to event_id
print(events[:10])

# Not it is also a good time to extract labels from the events
# With this our labels go from 0 to n_class - 1
labels = events[:, -1]
labels -= 1

In [ ]:
# Display EEG signal with some events
scal = dict(eeg=1e-3)     # EDIT ME!
plt.rcParams['figure.dpi'] = 150
raw_data.plot(events=events, event_color='red', event_id=event_id,
              scalings=scal, clipping=None, show_scrollbars=False, show_scalebars=False, 
              lowpass=40, highpass=2,          # EDIT ME!
              start=22, duration=40,  # EDIT ME!
              n_channels=32)
plt.show()

## Preprocessing Pipeline
We do not see a lot going on, for that we will have to move to the **frequency domain**, where we will be able to see and capture differences in frequency. Before that, we will have to pre-process the data. Different analysis require different pre-processing pipelines, and this time we will:

- Keep the relevant channels
- Band-pass filter the data
- Epoch the data

The only step that is new to this analysis is to keep a selection of channels. We will see an example first.

In [ ]:
# To keep a selection of channels, first select the channels we want to keep
ch_to_keep = ["Fp1", "Fp2"]

# Make a list of the channels to drop
ch_to_drop = list(set(raw_data.ch_names) - set(ch_to_keep))
print(ch_to_drop)  # All the channels except ch_to_keep

# Drop the rest of the channels using the drop_channels() function
raw_data = raw_data.drop_channels(ch_to_drop)

#### ❓ **Question** ❓: Load, pre-process, and epoch the data of a different subject using the functions presented until now. 

- Keep all the channels on the occipital (O) and parieto-occipital (PO) area
- Keep in mind the frequency of the stimuli for filtering (i.e., make sure to capture all the stimulation frequencies)
- Epoch between 0 and 2s, with a baseline of (0.2, 2)

In [ ]:
# Load the data 
question_file = 'P2_low_100.set'

data_path = os.path.join(data_dir, question_file)
raw_data = # HERE

# Get events and event_id
events, event_id = # HERE

# Drop all channels except selection
ch_to_keep = # HERE
ch_to_drop = # HERE

raw_data = # HERE

# Filter the data (notch and band-pass)
raw_data = # HERE (notch filter)
raw_data = # HERE (band-pass filter)

# Epoch the data
epochs = # HERE

#### 🔴 HINTS 🔴: Functions to use. Open if you need a reminder of the functions necessary for the avobe exercise

- load data: `mne.io.read_raw_eeglab`
- find events: `mne.events_from_annotations`
- drop channels: `raw_data.drop_channels`
- filter: `raw_data.filter`
- epoch: `mne.Epochs`

#### Additional 🔴 HINTS 🔴: Check if you feel lost

# II - Frequency analysis
We will now explore our data in the frequency domain, and use this information to tell apart the different stimulation frequencies. For that we will calculate the power spectral density (PSD) by calculating the Fourier Transform (FT).

The goal of this section is to explore the frequency domain to observe the SSVEP response, as it is the principle that will allow us to classify later.

## II-1 Using MNE functions

In [ ]:
# Necessary parameters for the FT
tmin = 0.
tmax = 2.
fmin = 1.
fmax = 90.
sfreq = epochs.info['sfreq']
print(event_id)
# In MNE, you can selection epoch based on labels
spectrum = epochs['12.000000'].compute_psd('welch',
                              n_fft=int(sfreq * (tmax - tmin)),
                              n_overlap=0, n_per_seg=None,
                              tmin=tmin, tmax=tmax,
                              fmin=fmin, fmax=fmax,
                              window='boxcar',
                              verbose=False)

psds, freqs = spectrum.get_data(return_freqs=True)
psds = 10*np.log10(psds) # convert to dB

In [ ]:
spectrum.plot()

🔴 HINTS 🔴  

Not that clear with all the electrodes at once but it seems that we do have a peak a 12Hz and then another one at ~24Hz so an harmonic.

## II-2 Using Scipy and Matplotlib

In [ ]:
# Extract epochs corresponding to a label

data = epochs['12.000000'].get_data()

f, psd = welch(data12, sfreq, nperseg=sfreq)
psd_trial = np.mean(psd, axis=0)


ch_names = epochs.info['ch_names']
fig, axes = plt.subplots(len(ch_names), figsize=(5, 3 * len(ch_names)))

for i, ch_name in enumerate(ch_names):
    #axes[i].stem(f, np.sqrt(psd_trial[i]), linefmt='b', markerfmt=" ", basefmt="-b")
    axes[i].plot(f,psd_trial[i])#linefmt='b', markerfmt=" ", basefmt="-b")
    axes[i].set_xlabel('Freq (Hz)')
    axes[i].set_ylabel('$\mu V^2/Hz$')
    #axes[i].set_yscale('log')
    axes[i].title.set_text(f'Electrode: {ch_names[i]}')
    
    axes[i].set_xticks(range(0, 40, 2))
    axes[i].set_xlim(0, 40)
    
fig.tight_layout()
plt.show()

🔴 HINTS 🔴  

We do have a clear peak on Oz !

#### ❓ **Question** ❓ Explore with other labels and accross electrodes. 

🔴 HINTS 🔴  
You can try to use a dB scale to counter-balance the $\frac{1}{f}$ law in the brain (more endougenous activy at low frequencies than higher ones).

- You can check the signature of any function (how to call it, arguments, documentation, etc.) adding a '?' after its name in a jupyter notebook cell, for example, try running `mne.events_from_annotations?` on a new cell
- You can check the list of all EEG channels in `raw_data.info['ch_names']`
- You can check the stimulation frequencies with the `event_id` dictionary.

# III - First classification using Canonical Correlation Analysis (CCA)
After having explored the frequency domain, we will now exploit this information to try to classify the different trials using various methods.

The first of them is Canonical Correlation Analysis (CCA). This method takes two random multivariate variables, $X$ and $Y$, and finds a transformation vector that makes the two of them be maximally correlated (this correlation is the so-called 'Canonical Correlation'). For more information, you can read the [CCA page on the Scikit-Learn documentation](https://scikit-learn.org/stable/modules/cross_decomposition.html#canonical-correlation-analysis).

The goal of this section is to lean how we can use CCA for our classification, and create a pipeline to classify the data from one of our participants. We will continue from the `epochs` object we created at the end of section I.

## What are we comparing our signal to?
As previously discussed, the stimulation were presented at certain frequencies. We also know that the brain activity in occipital areas shows a peak of activity at the frequency of stimulation. Knowing that, we can create 'artificial' signals that are perfect sinusoids at our target frequencies, for example:

In [ ]:
peak = 12  # Target frequency
trial_len = 1  # Length of the wave in seconds

# Time points for our wave 
t = np.arange(0, trial_len, 1 / sfreq)
sin = np.sin(2 * np.pi * peak * t)

plt.plot(t, sin)
plt.show()

And just like that, we created a perfect sinusoid at one of the stimulation frequencies! To make things better for CCA, each frequency will be compared with a pair of sine and cosine waves at their frequency, as well as pairs of waves corresponding to their harmonics (i.e. freq * N), like so:

In [ ]:
n_harmonics = 2

pairs = 1 + n_harmonics  # Include the frequency itself
harmonics = [i + 1 for i in range(pairs)]  # This gives us the list of numbers we need to multiply our taget freq by (including 1)
n_waves = pairs * 2  

all_waves = []
for i in harmonics:
    target_freq = i * peak

    sin = np.sin(2 * np.pi * (target_freq) * t)
    cos = np.cos(2 * np.pi * (target_freq) * t)

    all_waves.append(sin)
    all_waves.append(cos)

template = np.vstack(all_waves)  # (waves, samples)

We will call this the 'template' for that particular stimulation frequency. We need to create one for each frequency and then, we will compare each trial of real data to all the templates, and we will consider the one that shows the highest correlation the predicted class of the data. 

#### ❓ **Question** ❓: Make the templates for all classes in a single array

- Define the list of target frequencies
- Define the number of harmonics and total number of waves for each template
- Define the length of the waves, that needs to be equal to the length of the data epochs (in samples)
- Make an empty 3D array of shape (n_class, n_waves, trial_len)
- Iterate over peak frequencies and then over the harmonics to make the pairs of waves, stack all waves in one array and put them on the empty 3D array

In [ ]:
# Get target frequency list
event_id = epochs.event_id
peaks = # HERE

# Get number of harmonics and total waves
n_harmonics = 2
pairs = 1 + n_harmonics  # Including the frequency itself

harmonics = # HERE
n_waves = # HERE

# Get lenght of the wave (length of trials)
data = epochs.get_data()
trial_len = # HERE

# Make time points 't'
t = # HERE

# Create empty array
n_class = # HERE
ref_signals = # HERE

# Iterate over peaks (the index will be needed to add waves to ref_signals at the end
for class_idx, peak in enumerate(peaks):
    all_waves = []
    
    # Iterate over harmonics
    for i in harmonics:
        target_freq = # HERE 

        sin = # HERE
        cos = # HERE

        # Append the waves you just created 
        all_waves.append(sin)
        all_waves.append(cos)
        
    # Stack to get an array of shape (waves, samples)
    y = # HERE
    
    # Add to the empty array
    # HERE

#### 🔴 HINTS 🔴: Check here if you feel lost with the exercise

- In order to get the length of the trials, you can retrieve the data array with `epochs.get_data()`. The shape of this array is (n_trials, n_channels, n_samples)
- You can create a 3D array using the function `np.zeros()`. If you want an array of shape (3, 2, 4), you can make it with `np.zeros((3, 2, 4))`
- To assign a 2D array to the first dimension of a 3D array (i.e., set a (2, 4) `template` array) as the first element of your (3, 2, 4) `all_templates` array, you can do it with `all_templates[0, :, :] = template`
- As a reminder, the stimulation frequencies are stores in the `event_id` dict, that can be retrieved from your `epochs` with `epochs.event_id`

## CCA classification
Now we have our `ref_signals` dict. Inside of it are the perfect signals that we will use for all our frequency stimulations. Now the next question is: How do we exactly use these to classify our data? The process is as follows:

- We extract one epoch of data and find its true label from the `labels` array
- We iterate over our target frequencies and select the corresponding template
  - For each template, we fit a CCA model using our epoch and the template as $X$ and $Y$ variables
  - We transform them using the CCA model
  - We calculate the correlation between the transformed arrays and store it as the correlation for the corresponding class
- Finally, we take the argmax of the list with all the correlations as the predicted label for that trial

We will now see an example with one trial of data before proceeding to the final exercise of this section when you will implement the classification for all trials

In [ ]:
# First trial
ex_trial = data[0, ...]
ex_label = labels[0]

# Create the CCA model
cca = CCA(n_components=1, max_iter=1000)

# Empty list to store the correlations
corrs = []

# Iterate over classes
for class_idx in range(len(peaks)):
    # Get the corresponding template
    template = ref_signals[class_idx, :, :]
    
    # Fit CCA and transform
    cca.fit(ex_trial.T, template.T)
    x_scores, y_scores = cca.transform(ex_trial.T, template.T)
    
    # Get correlation
    corr_score = np.corrcoef(x_scores, y_scores, rowvar=False)[0, 1]
    corrs.append(corr_score)
    
pred = np.argmax(corrs)
print(f'True label: {ex_label}')
print(f'Predicted label: {pred}')

Now that we know how the process works for a single trial, it is time to do the same with all of them and put our newly acquired classifier to the test!

#### ❓ **Question** ❓: Build a CCA classifier and report the classification accuracy on one of our participants. You can use the same `epochs` and `ref_signals` from the previous exercises, we will focus only in building a loop for the classification.

- Create an empty list `pred` for the prediced classes for all the trials
- Create the CCA model at the beginning (no need to create a new one for each trial)
- Loop over the data trials. For each trial:
  - Repeat the single trial classification as before
  - Append the predicted label to your `pred` list
- Calculate and print the final accuracy

No hints for this one, I believe in you :D

In [ ]:
# Empty list for predictions
y_pred = []

# CCA model
cca = CCA(n_components=1, max_iter=1000)

# Loop over trials
n_trials, _, _ = data.shape

    # Iterate over classes

        # Fit CCA and transform

        # Get correlation
    
    # Append the label of the max correlation to the pred list

# Get accuracy

# Print accuracy
print(f'Total accuracy score: {acc_score}')

Well done! We have a good performance, accounting for the fact that we are using artificial data as templates. Research has proposed using CCA but creating templates from the data itself, more similar to a traditional machine learning approach (train -> create templates, test -> classification). Since demonstrating it would be too similar to what we just did, we are now moving to a classification approach specific to SSVEP data.

# IV - SSVEP classification using TRCA
Task-Related Component Analysis (TRCA) is an approach that share many similarities (conceptually, at least) with CCA. On this section, we will cover the process of TRCA and implement a classification pipeline that leverages it for our SSVEP data. We will not dive into the specifics of the approach, nor we will implement every step from scratch, so for those interested in the 'guts' of the model, refer to [the original TRCA paper](https://ieeexplore.ieee.org/abstract/document/7904641)

## How does TRCA work?
Simply put, TRCA creates a "class template" for each frequency by averaging data trials of the same class. Then, using the templates for all classes, TRCA computes linear filters that maximize the similarity between examples of the same class, while maximizing the differences between examples of different classes. This is similar to how CCA allowed us to transform our data and template so they had maximum correlation. From here, the classification process is similar to what we did with CCA:

- A data trial is extracted
- The process iterates over all classes
  - For each class, the data and the corresponding template are multiplied by the TRCA-filters
  - Then, the correlation between the two is calculated
- Finally, the class that holds the maximum correlation is considered the predicted class

Additionally, TRCA leverages a filtering approach called 'filterbank'. Templates, filters and data are divided into 'bands' at different frequencies. All the process described above is performed for all the specified frequency bands and the results are combined before the classification decision is taken.

## Classification
We provide you with a sklearn-compatible TRCA. Similarly to other classification algorithms that you have used, it uses a `fit()` method to calculate the templates and spatial filter and a `predict()` method that will give you the predicted labels for your test data.

## TRCA specifics
Before moving on to classification, we have to note some particularities for TRCA. First, the authors describe the need to omit the first 0.14 seconds (approximately) of data after each stimulus presentation. The reason is that this is the time that the information takes to reach the visual cortex. Also, we will perform the classification on the first second of data. With all this, we will take our data epochs and select a slice from 0.14 to 1.14 seconds. Finally, we will take this chance to specify the number of bands for filterbank, as well as the downsample parameter for TRCA

In [ ]:
init_delay = 0.14
epoch_len = 1.
n_fbands = 4
downsample = 2  # The original study downsampled to 250Hz, so we will do the same

# Get t_min and t_max in samples to slice the data
t_min = int(init_delay * sfreq) + 1
t_max = int(t_min + epoch_len * sfreq)

# Slice the data
data_slice = data[..., t_min:t_max]

#### ❓ **Question** ❓: Classify the data using TRCA.

Now that we have an intuition about how TRCA works, the only thing left for us to do is to build a classification pipeline. For that, you will have to:

- Start from the same `epochs` object we have been using so far
- Split the data into train and test using an aprox of 33% testing data
- Fit the train data and test on the testing data
- Compute and print the accuracy of your classification

When creating the TRCA model, you will have to provide the following arguments: `sfreq`, `n_fbands`, `peaks` and `downsample`

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = # HERE

# Classifier
clf = # HERE

# Fit and predict
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

acc = # HERE
print(f'TRCA accuracy: {acc}')